In [1]:
!pip install plotly

     |████████████████████████████████| 6.8MB 7.5MB/s eta 0:00:01
  Stored in directory: /home/ahmed/.cache/pip/wheels/d7/a9/33/acc7b709e2a35caa7d4cae442f6fe6fbf2c43f80823d46460c
Successfully built retrying


In [26]:
import pandas as pd
import os
import json

import pymysql
with open(os.getcwd()+'/config/config.json') as config_file:
        params = json.load(config_file)

def get_db_connection():
    with open(os.getcwd()+'/config/config.json') as config_file:
        user = params['db_user']
        passw = params['dbpassword']  
        host =  params['hostname']
        port = params['port']
        database=params['databasename']
        conn = pymysql.connect(host,user,passw,database )
        return conn
    
    
conn = get_db_connection()
query = """ SELECT SUBSTRING(O_ORDERDATE , 1,4) TX_YEAR, SUBSTRING(O_ORDERDATE , 6,2)  TX_MONTH , 
SUM(LINEITEM_REVENUE) total_revenue  FROM ORDER_ITEM_FACT  GROUP BY  SUBSTRING(O_ORDERDATE , 1,4), SUBSTRING(O_ORDERDATE , 6,2)    ORDER BY 1,2;
;"""
ds1 = pd.read_sql(query, conn  )
ds1["total_revenue"] = ds1.total_revenue.astype(float)
df_2016 = ds1.query("TX_YEAR=='2016'")
df_2017= ds1.query("TX_YEAR=='2017'")
df_cd = pd.merge(df_2016, df_2017, how='inner', on = 'TX_MONTH')
df_cd

,TX_YEAR_x,TX_MONTH,total_revenue_x,TX_YEAR_y,total_revenue_y
0,2016,01,3.730390e+09,2017,2.685772e+09
1,2016,02,3.504858e+09,2017,2.641949e+09
2,2016,03,3.869116e+09,2017,2.600459e+09
3,2016,04,3.582271e+09,2017,2.659999e+09
4,2016,05,3.742992e+09,2017,2.892257e+09
5,2016,06,3.551666e+09,2017,2.610612e+09
6,2016,07,3.634146e+09,2017,2.649783e+09
7,2016,08,3.073819e+09,2017,2.632730e+09
8,2016,09,2.895060e+09,2017,2.726494e+09
9,2016,10,2.600950e+09,2017,2.890236e+09


In [27]:
import plotly.offline as py
import pandas as pd

import plotly.graph_objects as go

 
fig = go.Figure()
fig.add_trace(go.Scatter(
                x=df_2016['TX_MONTH'],
                y=df_2016['total_revenue'],
                name="Year 2016",
                line_color='deepskyblue',
                opacity=0.8))

fig.add_trace(go.Scatter(
                x=df_2017['TX_MONTH'],
                y=df_2017['total_revenue'],
                name="Year 2017",
                line_color='dimgray',
                opacity=0.8))




# Use date string to set xaxis range
fig.update_layout(xaxis_range=['01','12' ],
                  title_text="Manually Set Date Range")
fig.show()